# Installing libraries

In [155]:
#!pip install transformers sentence-transformers faiss-cpu torch
#!pip install requests
#!pip install together

# Imports 

In [156]:
from together import Together
import re
from sentence_transformers import SentenceTransformer
import faiss
from together import Together
import numpy as np

# The real experiments

In [157]:
def clean_text(text):
    """Cleans and normalizes the text."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(\n\s*)+', '\n', text)
    return text.strip()

def prepare_knowledge_base(text):
    """Splits the document into manageable chunks."""
    text = clean_text(text)
    paragraphs = re.split(r'(§\s*\d+\..*?)(?=§|\Z)', text)
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    final_chunks = []
    for p in paragraphs:
        if len(p.split()) > 100: 
            sentences = re.split(r'(?<=[.!?])\s+', p)
            chunk = ''
            for s in sentences:
                if len(chunk.split()) + len(s.split()) < 100:
                    chunk += ' ' + s
                else:
                    if chunk:
                        final_chunks.append(chunk.strip())
                    chunk = s
            if chunk:
                final_chunks.append(chunk.strip())
        else:
            final_chunks.append(p)
    return final_chunks

with open("../input/regulamin-agh/regulamin_agh.txt", "r", encoding="utf-8") as file:
    document_text = file.read()

knowledge_base = prepare_knowledge_base(document_text)

retriever = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = retriever.encode(knowledge_base)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

def prepare_training_data(queries, answers, knowledge_base, retriever, index):
    training_data = []
    for query, answer in zip(queries, answers):
        print(f"\nProcessing query: {query}")
        query_embedding = retriever.encode([query])
        _, indices = index.search(query_embedding, k=2)
        contexts = [knowledge_base[i] for i in indices[0]]
        context = " ".join(contexts)
        print(f"Retrieved context: {context}")

        training_data.append({
            "query": query,
            "context": context,
            "answer": answer
        })
    return training_data

training_data = prepare_training_data(queries, answers, knowledge_base, retriever, index)

print("\nTraining Data:")
for data in training_data:
    print(f"\nQuery: {data['query']}")
    print(f"Context: {data['context']}")
    print(f"Answer: {data['answer']}")

Batches:   0%|          | 0/10 [00:00<?, ?it/s]


Processing query: What are the rules for taking a leave of absence?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved context: The leave a s referred to in section 1 point 1 shall be granted for the period until a child's birth, however, if a leave ends during the semester, it shall be granted until the end of this semester of studies. 5. The leave as referred to in section 1 point 2 shall be gr anted for a period of up to one year, however, if the leave ends during the semester, it shall be granted until the end of this semester of studies. 6. § 20. LEAVE FORM CLASSES 1. The Dean of the Faculty shall grant a student a leave from classes at the University for a period of no longer than one year, subject to section 4 and 5: a) a maternity leave – in case of a pregnant student; b) a parental leave – in case of a student being a parent in order to take care of a child. 2.

Processing query: What are the conditions for passing a semester?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieved context: § 14. GRADE POINT AVERAGE 1. Grade point average (GPA) for a given period of studies shall be a measure of a student academic progress in that period and may be determined for one semester, several semesters and for the entire period of studies. 2. In each case, grade point average shall be a weighted average of all final grades for courses completed in a given period. Weight coefficients shall be ECTS credits assigned to courses: 𝑔𝑟𝑎𝑑𝑒 𝑝𝑜𝑖𝑛𝑡 𝑎𝑣𝑒𝑟𝑎𝑔𝑒 = Ʃ 𝐸𝐶𝑇𝑆 𝑐𝑟𝑒𝑑𝑖𝑡𝑠 𝑎𝑠𝑠𝑖𝑔𝑛𝑒𝑑 𝑡𝑜 𝑎 𝑐𝑜𝑢𝑟𝑠𝑒 ∗𝑓𝑖𝑛𝑎𝑙 𝑔𝑟𝑎𝑑𝑒 𝑓𝑜𝑟 𝑎 𝑐𝑜𝑢𝑟𝑠𝑒 Ʃ 𝐸𝐶𝑇𝑆 𝑐𝑟𝑒𝑑𝑖𝑡𝑠 𝑓𝑜𝑟 𝑐𝑜𝑢𝑟𝑠𝑒𝑠 𝑡𝑎𝑘𝑒𝑛 𝑖𝑛𝑡𝑜 𝑎𝑐𝑐𝑜𝑢𝑛𝑡 𝑤ℎ𝑒𝑛 𝑐𝑎𝑙𝑐𝑢𝑙𝑎𝑡𝑖 𝑛𝑔 𝑔𝑟𝑎𝑑𝑒 𝑝𝑜𝑖𝑛𝑡 𝑎𝑣𝑒𝑟𝑎𝑔𝑒 3. Grade point average shall b e established to two decimal places, without rounding. 4. Additional courses shall be excluded from the calculation of grade point average, i.e. those which a student studies with the consent of the Dean of the Faculty, beyond the curriculum, and those comple ted without a final grade.

Processing query: What are the rules for repeating a subjec

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved context: § 18. REPEATING A MODULE OR A COURSE 1. In case of failure to meet conditions for completing a module or a course in a given semester, a student shall be required to repeat it within the time limit specified by the Dean of the Faculty. A request in this matter shall be submitted to the Dean of the Faculty. 2. When repeating a semester of studies, a student shall be required to study according to the curriculum applicable for a given year of education in the semester f or which he/she was registered. 9. When giving a consent to repeat a semester of studies, if necessary, the Dean of the Faculty shall establish in writing curriculum differences and deadlines for their completion. 10.

Processing query: What are the requirements for graduation?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved context: § 27. GRADUATION 1. In order to graduate and to obtain a graduation diploma in a given field of study, level and profile, a student shall be required to: 1) achieve all assumed learning outcomes specified in the curriculum; 2) complete all modules provided for in the curriculum; 3) obtain a number of ECTS credit s required by the curriculum; 4) in case of first -cycle degree programmes – submit a diploma project, and in case of second -cycle degree programmes – submit a diploma thesis; 5) pass a diploma examination. 2. § 4. ADMISSION TO STUDIES 1.

Training Data:

Query: What are the rules for taking a leave of absence?
Context: The leave a s referred to in section 1 point 1 shall be granted for the period until a child's birth, however, if a leave ends during the semester, it shall be granted until the end of this semester of studies. 5. The leave as referred to in section 1 point 2 shall be gr anted for a period of up to one year, however, if the leave ends during 

In [158]:
queries = [
    "What are the rules for taking a leave of absence?",
    "What are the conditions for passing a semester?",
    "How is the grade point average calculated?",
    "What are the rules for repeating a subject?",
    "What are the requirements for graduation?"
]

answers = [
    "According to §20, students may be granted leave for maternity, parental, health, special circumstances, professional reasons, or without justification, for a period not exceeding one year.",
    "According to §17, passing a semester requires earning 27-33 ECTS credits and completing all modules in the study plan. Students may register for the next semester with an ECTS credit deficit of 6-15 points.",
    "According to §14, the grade point average is calculated as the weighted average of all final grades, where weights are the ECTS credits assigned to each course.",
    "According to §18, if a student fails a subject, they must repeat it in a period designated by the Dean. Repeating requires re-attending classes and paying associated fees.",
    "According to §27, graduation requires achieving learning outcomes, completing all modules, earning the required ECTS credits, submitting a thesis or project, and passing the diploma examination."
]

In [161]:
client = Together(api_key="api_key")

def retrieve_context(query, retriever, index, knowledge_base, top_k=3):
    query_embedding = retriever.encode([query])
    scores, indices = index.search(query_embedding.reshape(1, -1), top_k)
    contexts = []
    for i, score in zip(indices[0], scores[0]):
        if score < 20:  # Filter by relevance score
            contexts.append(knowledge_base[i])
    print(f"Retrieved contexts for query '{query}': {contexts}")
    return contexts

def evaluate_predictions(predictions, true_answers):
    def calculate_f1(pred, true):
        pred_tokens = pred.strip().lower().split()
        true_tokens = true.strip().lower().split()
        
        if not pred_tokens or not true_tokens:
            return 0.0, 0.0, 0.0
        
        common = set(pred_tokens) & set(true_tokens)
        precision = len(common) / len(pred_tokens) if pred_tokens else 0
        recall = len(common) / len(true_tokens) if true_tokens else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return f1, precision, recall

    f1_scores = []
    precisions = []
    recalls = []
    
    for pred, true in zip(predictions, true_answers):
        f1, precision, recall = calculate_f1(pred, true)
        f1_scores.append(f1)
        precisions.append(precision)
        recalls.append(recall)

    return {
        "F1 Score": sum(f1_scores) / len(f1_scores),
        "Precision": sum(precisions) / len(precisions),
        "Recall": sum(recalls) / len(recalls),
    }


def generate_with_vision_free(query, client, model="meta-llama/Llama-Vision-Free"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": query}],
            max_tokens=100,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.1,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=False
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating response: {e}")
        return ""

def generate_with_hipporag_and_vision_free(query, retriever, index, knowledge_base, client, model="meta-llama/Llama-Vision-Free"):
    try:
        contexts = retrieve_context(query, retriever, index, knowledge_base, top_k=3) 
        
        prompt = f"""Answer the following question based on the provided context. Be specific and concise.
Context: {' '.join(contexts)}
Question: {query}
Key information to include: {', '.join(key_phrases_from_context(contexts))}
Answer:"""
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a precise QA assistant. Use context to answer questions accurately."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  
            temperature=0.2,  
            top_p=0.9,
            repetition_penalty=1.2,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=False
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating response: {e}")
        return ""

def key_phrases_from_context(contexts):
    phrases = []
    for context in contexts:
        important_sentences = [s.strip() for s in context.split('.') if len(s.strip()) > 10]
        phrases.extend(important_sentences[:2]) 
    return phrases[:3] 

def prepare_training_data(queries, answers, knowledge_base, retriever, index):
    training_data = []
    for query, answer in zip(queries, answers):
        contexts = retrieve_context(query, retriever, index, knowledge_base, top_k=2)
        context = " ".join(contexts)

        training_data.append({
            "vision_free_prompt": query,
            "hipporag_prompt": f"Context: {context}\nQuestion: {query}",
            "completion": answer
        })
    return training_data

def train_models(training_data, client, model="meta-llama/Llama-Vision-Free", epochs=3):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for data in training_data:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": data["vision_free_prompt"]},
                         {"role": "assistant", "content": data["completion"]}],
                max_tokens=100,
                temperature=0.3,
                stop=["<|eot_id|>", "<|eom_id|>"]
            )
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": data["hipporag_prompt"]},
                         {"role": "assistant", "content": data["completion"]}],
                max_tokens=100,
                temperature=0.3,
                stop=["<|eot_id|>", "<|eom_id|>"]
            )

training_data = prepare_training_data(queries, answers, knowledge_base, retriever, index)
train_models(training_data, client)

vision_free_predictions = [generate_with_vision_free(query, client) for query in queries]
print("\nVision-Free Predictions:")
for pred, true in zip(vision_free_predictions, answers):
    print(f"\nPrediction: {pred}")
    print(f"True: {true}")

hipporag_predictions = [generate_with_hipporag_and_vision_free(query, retriever, index, knowledge_base, client) for query in queries]
print("\nHippoRAG Predictions:")
for pred, true in zip(hipporag_predictions, answers):
    print(f"\nPrediction: {pred}")
    print(f"True: {true}")

print("\nEvaluation Results:")
print("Vision-Free:", evaluate_predictions(vision_free_predictions, answers))
print("HippoRAG:", evaluate_predictions(hipporag_predictions, answers))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved contexts for query 'What are the rules for taking a leave of absence?': ["The leave a s referred to in section 1 point 1 shall be granted for the period until a child's birth, however, if a leave ends during the semester, it shall be granted until the end of this semester of studies. 5. The leave as referred to in section 1 point 2 shall be gr anted for a period of up to one year, however, if the leave ends during the semester, it shall be granted until the end of this semester of studies. 6.", '§ 20. LEAVE FORM CLASSES 1. The Dean of the Faculty shall grant a student a leave from classes at the University for a period of no longer than one year, subject to section 4 and 5: a) a maternity leave – in case of a pregnant student; b) a parental leave – in case of a student being a parent in order to take care of a child. 2.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved contexts for query 'What are the conditions for passing a semester?': ['A student, with the consent of the Dean of the Faculty, may also pursue classes not covered by the curriculum for a studied field of study, but once they are placed in a semester timetable a student shall be required to complete them, in accordance with the requirements set out in the Study Regulations, the curriculum, and by a teacher responsible for the course in the syllabus as referred to in § 10 section 6.', 'A semester timetable shall include: 1) all compulsory and elective modules that a student intends to pursue and complete in a given semester, provide d that the assigned number of ECTS credits may not be less than a number of ECTS credits provided for in the study plan for a given semester, ranging from 27 to 33 ECTS credits, and may not be less than 60 ECTS credits for a given year of study, if it last s for two semesters, subject to']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieved contexts for query 'How is the grade point average calculated?': ['§ 14. GRADE POINT AVERAGE 1. Grade point average (GPA) for a given period of studies shall be a measure of a student academic progress in that period and may be determined for one semester, several semesters and for the entire period of studies. 2. In each case, grade point average shall be a weighted average of all final grades for courses completed in a given period.', 'Weight coefficients shall be ECTS credits assigned to courses: 𝑔𝑟𝑎𝑑𝑒 𝑝𝑜𝑖𝑛𝑡 𝑎𝑣𝑒𝑟𝑎𝑔𝑒 = Ʃ 𝐸𝐶𝑇𝑆 𝑐𝑟𝑒𝑑𝑖𝑡𝑠 𝑎𝑠𝑠𝑖𝑔𝑛𝑒𝑑 𝑡𝑜 𝑎 𝑐𝑜𝑢𝑟𝑠𝑒 ∗𝑓𝑖𝑛𝑎𝑙 𝑔𝑟𝑎𝑑𝑒 𝑓𝑜𝑟 𝑎 𝑐𝑜𝑢𝑟𝑠𝑒 Ʃ 𝐸𝐶𝑇𝑆 𝑐𝑟𝑒𝑑𝑖𝑡𝑠 𝑓𝑜𝑟 𝑐𝑜𝑢𝑟𝑠𝑒𝑠 𝑡𝑎𝑘𝑒𝑛 𝑖𝑛𝑡𝑜 𝑎𝑐𝑐𝑜𝑢𝑛𝑡 𝑤ℎ𝑒𝑛 𝑐𝑎𝑙𝑐𝑢𝑙𝑎𝑡𝑖 𝑛𝑔 𝑔𝑟𝑎𝑑𝑒 𝑝𝑜𝑖𝑛𝑡 𝑎𝑣𝑒𝑟𝑎𝑔𝑒 3. Grade point average shall b e established to two decimal places, without rounding. 4. Additional courses shall be excluded from the calculation of grade point average, i.e. those which a student studies with the consent of the Dean of the Faculty, beyond the curriculum, and those comple ted without a final grade.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved contexts for query 'What are the rules for repeating a subject?': ['§ 18. REPEATING A MODULE OR A COURSE 1. In case of failure to meet conditions for completing a module or a course in a given semester, a student shall be required to repeat it within the time limit specified by the Dean of the Faculty. A request in this matter shall be submitted to the Dean of the Faculty. 2.', 'When repeating a semester of studies, a student shall be required to study according to the curriculum applicable for a given year of education in the semester f or which he/she was registered. 9. When giving a consent to repeat a semester of studies, if necessary, the Dean of the Faculty shall establish in writing curriculum differences and deadlines for their completion. 10.', 'In exceptional, particularly justified cases, the Dean of the F aculty may, once at each level of studies, agree to repeat a semester of studies once again. 5. The rule referred to in section 3 and 4, shall not apply to part 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved contexts for query 'What are the requirements for graduation?': ['§ 27. GRADUATION 1. In order to graduate and to obtain a graduation diploma in a given field of study, level and profile, a student shall be required to: 1) achieve all assumed learning outcomes specified in the curriculum; 2) complete all modules provided for in the curriculum; 3) obtain a number of ECTS credit s required by the curriculum; 4) in case of first -cycle degree programmes – submit a diploma project, and in case of second -cycle degree programmes – submit a diploma thesis; 5) pass a diploma examination. 2.', '§ 4. ADMISSION TO STUDIES 1.', 'In order to graduate and obtain a graduation diploma in a given field, level and profile, a student shall be required to obtain: 1) for the first -cycle degree programme (bachelor programmes) – at least 180 ECTS credits; 2) for the first -cycle degree pr ogramme (engineer programmes) – at least 210 ECTS credits; 3) for the second -cycle degree programme (master 